In [9]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [10]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [11]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [12]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_19_9_4,0.998094,0.993332,0.994878,0.993931,0.561198,1.600328,0.666879,1.161060,"Hidden Size=[30, 15], regularizer=0.01, learni..."
1,model_19_9_3,0.998079,0.994222,0.995426,0.994697,0.565645,1.386740,0.595462,1.014370,"Hidden Size=[30, 15], regularizer=0.01, learni..."
2,model_19_9_5,0.998060,0.992322,0.994301,0.993075,0.571200,1.842625,0.742030,1.324699,"Hidden Size=[30, 15], regularizer=0.01, learni..."
3,model_19_9_2,0.998000,0.994942,0.995933,0.995338,0.588905,1.213778,0.529550,0.891790,"Hidden Size=[30, 15], regularizer=0.01, learni..."
4,model_19_9_6,0.997989,0.991235,0.993707,0.992163,0.592019,2.103405,0.819309,1.499125,"Hidden Size=[30, 15], regularizer=0.01, learni..."
6,model_4_9_23,0.997974,0.988570,0.987562,0.988116,0.596656,1.287493,2.620675,1.914870,"Hidden Size=[15, 5], regularizer=0.1, learning..."
7,model_4_9_22,0.997971,0.988565,0.987656,0.988171,0.597548,1.288159,2.600956,1.905943,"Hidden Size=[15, 5], regularizer=0.1, learning..."
8,model_4_9_21,0.997965,0.988557,0.987758,0.988232,0.599034,1.288983,2.579306,1.896193,"Hidden Size=[15, 5], regularizer=0.1, learning..."
9,model_4_9_20,0.997958,0.988548,0.987871,0.988298,0.601293,1.289997,2.555531,1.885544,"Hidden Size=[15, 5], regularizer=0.1, learning..."
10,model_4_9_19,0.997947,0.988537,0.987995,0.988370,0.604551,1.291206,2.529508,1.873941,"Hidden Size=[15, 5], regularizer=0.1, learning..."
